Technohack virtual internship : Task2
<hr/>

Dataset: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset

<br/>
Task Details  :Use a dataset of emails and build a model thatcan identify spam emails

Step 1 : Importing all the required modules

In [41]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
"""from spellchecker import SpellChecker"""
"""from IPython.display import clear_output"""

'from IPython.display import clear_output'

Downloading some required corpus of words from nltk

In [42]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\resha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\resha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
"""nltk.download('wordnet')"""

"nltk.download('wordnet')"

In [45]:
"""nltk.download('omw-1.4')"""

"nltk.download('omw-1.4')"

In [46]:
"""nltk.download('words')"""

"nltk.download('words')"

some of the statements are commented as they are already downloaded

Step 2 : Importing the dataset

In [47]:
maindf  = pd.read_csv("./spam.csv", encoding = "ISO-8859-1", engine = "python")
maindf.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Removing the extra columns

In [48]:
maindf.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"], axis=1,inplace=True)
maindf.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Checking for null values

In [49]:
maindf.isnull().sum()

v1    0
v2    0
dtype: int64

Removing special characters (.,-,_ ....)

In [50]:
def remove_special_characters(text):
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return cleaned_text

maindf['v2'] = maindf['v2'].apply(remove_special_characters)
maindf.head()

,v1,v2
0,ham,Go until jurong point crazy Available only in ...
1,ham,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor U c already then say
4,ham,Nah I dont think he goes to usf he lives aroun...


Lemmatization: </br>
Lemmatization is the process of reducing words to their base or dictionary form, known as the lemma. In lemmatization, words are transformed to their canonical or morphological root form, which is linguistically valid and meaningful. The resulting lemma represents the base meaning of the word.</br>
</br>
Stemming:</br>
Stemming is the process of reducing words to their root or stem form by removing suffixes or prefixes. Stemming operates based on simple heuristics and pattern matching, without considering the context or part of speech of the word.

In [51]:
def perform_lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in text.split()]
    return ' '.join(lemmatized_words)

def perform_stemming(text):
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in text.split()]
    return ' '.join(stemmed_words)

maindf['v2'] = maindf['v2'].apply(perform_lemmatization)
maindf['v2'] = maindf['v2'].apply(perform_stemming)
maindf.head()

,v1,v2
0,ham,go until jurong point crazi avail onli in bugi...
1,ham,ok lar joke wif u oni
2,spam,free entri in 2 a wkli comp to win fa cup fina...
3,ham,u dun say so earli hor u c alreadi then say
4,ham,nah i dont think he go to usf he life around h...


In [52]:
len(maindf)

5572

Performing spellcheck. For this we are using SepllChecker Module

In [53]:
spell = SpellChecker()
def correct_spelling(text):
    if text['v2'] is None:
        return None
    outtext = ''
    print(text.name, end=" ")
    if text.name %10 == 0:
        clear_output()
    for word in text['v2'].split():
        txt = spell.correction(word)
        if txt:
            outtext+= " " + spell.correction(word)
        else:
            outtext+= " " + word
    return outtext
maindf['v2']=maindf.apply(correct_spelling, axis=1)

'spell = SpellChecker()\ndef correct_spelling(text):\n    if text[\'v2\'] is None:\n        return None\n    outtext = \'\'\n    print(text.name, end=" ")\n    if text.name %10 == 0:\n        clear_output()\n    for word in text[\'v2\'].split():\n        txt = spell.correction(word)\n        if txt:\n            outtext+= " " + spell.correction(word)\n        else:\n            outtext+= " " + word\n    return outtext\nmaindf[\'v2\']=maindf.apply(correct_spelling, axis=1)'

Since the dataset is large we will save the spellchecked dataframe to separate csv file so that we do not have to redo this step on every load

In [54]:
"""maindf.to_csv('spellchecked.csv', index=False)"""

"maindf.to_csv('spellchecked.csv', index=False)"

after spell check

In [55]:
maindfspellchecked = pd.read_csv("./spellchecked.csv")
maindfspellchecked.head()

,v1,v2
0,ham,go until wrong point crazy avail only in bug ...
1,ham,ok lar joke wif u oni
2,spam,free entry in 2 a will comp to win fa cup fin...
3,ham,u dun say so early hor u i already then say
4,ham,nah i dont think he go to us he life around h...


before spell check

In [56]:
maindf.head()

,v1,v2
0,ham,go until jurong point crazi avail onli in bugi...
1,ham,ok lar joke wif u oni
2,spam,free entri in 2 a wkli comp to win fa cup fina...
3,ham,u dun say so earli hor u c alreadi then say
4,ham,nah i dont think he go to usf he life around h...


Removing Stop Words </br>
Stop words are common words that are often considered insignificant or carry little meaning in the context of natural language processing and text analysis tasks.  (e.g., "a," "an," "the") </br>
For this we will use NLTK stop word corpus

In [57]:
stop_words = set(stopwords.words('english'))

In [58]:
def removestopword(text):
    words = nltk.word_tokenize(str(text))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)


maindf['v2']= maindf['v2'].apply(removestopword)
maindfspellchecked['v2'] = maindfspellchecked['v2'].apply(removestopword)

In [59]:
maindf.head()

,v1,v2
0,ham,go jurong point crazi avail onli bugi n great ...
1,ham,ok lar joke wif u oni
2,spam,free entri 2 wkli comp win fa cup final tkt 21...
3,ham,u dun say earli hor u c alreadi say
4,ham,nah dont think go usf life around though


Step 3 : Model Building

    Models used
    
    Naive Bayes Classifier

    Support Vector Machines (SVM)

X , Y Split

In [73]:
Y_normal = maindf['v1']
X_normal = maindf['v2']
X_spellchecked = maindfspellchecked['v2']
Y_spellchecked = maindfspellchecked['v1']

Lable encoding for y values as they are text values

In [61]:
normalencder = LabelEncoder()
Y_normal = normalencder.fit_transform(Y_normal)
spellcheckedencoder = LabelEncoder()
Y_spellchecked = spellcheckedencoder.fit_transform(Y_spellchecked)

Train-Test split

In [62]:
X_train_normal, X_test_normal, y_train_normal, y_test_normal = train_test_split(X_normal, Y_normal, test_size=0.2, random_state=42)
X_train_spellchecked, X_test_spellchecked, y_train_spellchecked, y_test_spellchecked = train_test_split(X_spellchecked, Y_spellchecked, test_size=0.2, random_state=42)

Since we are working on text data the data and computers only understand numbers we need to convert the text to numerical data such that we can use that data for classification purpose </br>
To do so we will use TF-IDF algorithm</br>
TF-IDF stands for Term Frequency-Inverse Document Frequency. It is a numerical representation technique used in natural language processing to quantify the importance of a term (word) in a document relative to a corpus (a collection of documents).

In [63]:
normalvectorizer = TfidfVectorizer()
spellcheckedvectorizer = TfidfVectorizer()
X_normal_tfidf_train = normalvectorizer.fit_transform(X_train_normal)
X_spellchecked_tfidf_train = spellcheckedvectorizer.fit_transform(X_train_spellchecked)
X_normal_tfidf_test = normalvectorizer.transform(X_test_normal)
X_spellchecked_tfidf_test = spellcheckedvectorizer.transform(X_test_spellchecked)

Now lets test Naive-bayes model for our data without spell check

In [64]:
nb_clasifier = MultinomialNB()
nb_clasifier.fit(X_normal_tfidf_train,y_train_normal)
y_pred = nb_clasifier.predict(X_normal_tfidf_test)

In [65]:
print(classification_report(y_test_normal,y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.71      0.83       150

    accuracy                           0.96      1115
   macro avg       0.98      0.85      0.90      1115
weighted avg       0.96      0.96      0.96      1115



Now lets test Naive-bayes model for our data with spell check

In [66]:
nb_clasifier = MultinomialNB()
nb_clasifier.fit(X_spellchecked_tfidf_train,y_train_spellchecked)
y_pred = nb_clasifier.predict(X_spellchecked_tfidf_test)

In [67]:
print(classification_report(y_test_spellchecked,y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.73      0.85       150

    accuracy                           0.96      1115
   macro avg       0.98      0.87      0.91      1115
weighted avg       0.97      0.96      0.96      1115



This is already a good result we can see precision of .96 and but recal for 1 is not satisfactory

So lets test support vector classifier

In [68]:
normalsvc = SVC()
normalsvc.fit(X_normal_tfidf_train,y_train_normal)
y_pred = normalsvc.predict(X_normal_tfidf_test)

In [69]:
print(classification_report(y_test_normal,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.98      0.84      0.91       150

    accuracy                           0.98      1115
   macro avg       0.98      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [71]:
spellcheckedsvc = SVC()
spellcheckedsvc.fit(X_spellchecked_tfidf_train,y_train_spellchecked)
y_pred = spellcheckedsvc.predict(X_spellchecked_tfidf_test)

In [72]:
print(classification_report(y_test_normal,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.98      0.85      0.91       150

    accuracy                           0.98      1115
   macro avg       0.98      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



As you can see we have recal of 1 and .85 and high precision as well

lets test this model on user input data

In [90]:
userinput = pd.Series(["how are you","free product"])
userinput = spellcheckedvectorizer.transform(userinput)

print(spellcheckedsvc.predict(userinput))

[0 1]
